<a href="https://colab.research.google.com/github/Tiabet/DACON_WebClick/blob/main/WebClick_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
train = pd.read_parquet('/content/drive/MyDrive/data/train.parquet')
test =  pd.read_parquet('/content/drive/MyDrive/data/test.parquet')

In [ ]:
# 동일한 결과 보장을 위해 Seed값을 고정합니다
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed를 42로 고정

In [ ]:
test.drop('ID', axis = 1, inplace = True)
train.drop('ID', axis = 1, inplace = True)
train.head()

,Click,F01,F02,F03,F04,F05,F06,F07,F08,F09,...,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39
0,1,NSLHFNS,AVKQTCL,DTZFPRW,114.0,ISVXFVA,1,PQZBVMG,LPYPUNA,IZYJZDA,...,NZGEZLW,GTISJWW,380.0,2.0,AXQFZWC,IRUDRFB,NaN,TFJMLCZ,0.0,AURZYDY
1,0,VGIVWZQ,LSUSMVO,PQGWFJZ,26.0,NFRVLWS,43,IMPIGJT,MIGYEEG,NGODWIN,...,NZGEZLW,GTISJWW,466.0,1.0,DRVVDHZ,IRUDRFB,19.0,AUGTURV,0.0,LUZRMLU
2,0,JCDXFYU,PILDDJU,IAGJDOH,119.0,LFPUEOV,0,FFUTIRZ,OFKQGTY,BEZTQIO,...,VHXETCF,KHZNEZF,197.0,0.0,QMOULXS,IRUDRFB,8.0,ZVSTLNM,0.0,MHBRSQK
3,1,PSMFWTP,ZYAVJHP,None,15.0,ATQPZSJ,26,ZDTZNSB,THBWWCD,LTETYBG,...,IVIRTPR,GTISJWW,8640.0,0.0,IZLJUJS,IRUDRFB,14.0,ZBSRLCQ,0.0,GAZBSSZ
4,0,SLCRICD,QPQWGXA,None,13.0,CHZGJZR,20,PQZBVMG,MIGYEEG,LJBQPJW,...,NZGEZLW,WHSRKIM,41774.0,0.0,BHBIZCL,IRUDRFB,13.0,QHYLSBX,0.0,QTATWAY


의미없는 카테고리들 삭제

In [ ]:
# Step 1: Identify the categories in the train set that appear 100,000 times or more
category_thresholds = {}
for column in train.columns:
    if column != 'Click':
        # Calculate the value counts for each column
        value_counts = train[column].value_counts()

        # Find values that appear 100,000 times or more
        categories_to_keep = value_counts[value_counts >= 50000].index

        # Store these categories for each column
        category_thresholds[column] = categories_to_keep

        # Replace values in train set that appear less than 100,000 times with NaN
        train.loc[~train[column].isin(categories_to_keep), column] = pd.NA

# Step 2: Apply these categories to the test set
for column in test.columns:
    if column != 'Click' and column in category_thresholds:
        categories_to_keep = category_thresholds[column]

        # Replace values in test set that are not in the categories_to_keep with NaN
        test.loc[~test[column].isin(categories_to_keep), column] = pd.NA

결측치채우기

In [ ]:
import numpy as np

# Function to fill NaNs based on existing distribution
def fill_na_with_distribution(column):
    counts = column.dropna().value_counts(normalize=True)
    nans_to_fill = column.isna().sum()
    fill_values = np.random.choice(counts.index, size=nans_to_fill, p=counts.values)
    column.loc[column.isna()] = fill_values
    return column

# Apply the function to each column in DataFrame
train = train.apply(fill_na_with_distribution, axis=0)
test = test.apply(fill_na_with_distribution, axis=0)

샘플링

In [ ]:
train = train.groupby('Click').apply(lambda x: x.sample(min(len(x), len(test))))

In [ ]:
train_x = train.drop('Click', axis = 1)
train_y = train['Click']

In [ ]:
del train

In [ ]:
# Label encode categorical columns
def label_encode_columns(train_df, test_df):
    le = LabelEncoder()
    for column in train_df.columns:
        if train_df[column].dtype == 'object' or isinstance(train_df[column].dtype, pd.CategoricalDtype):
            # Fit the LabelEncoder on the combined data to ensure consistency
            combined_data = pd.concat([train_df[column], test_df[column]], axis=0).astype(str)
            le.fit(combined_data)
            train_df[column] = le.transform(train_df[column].astype(str))
            test_df[column] = le.transform(test_df[column].astype(str))
    return train_df, test_df

# Apply label encoding to train_x and test_x
train_x, test_x = label_encode_columns(train_x, test)

In [ ]:
train_x.to_parquet('drive/MyDrive/data/train_v3.parquet', compression = 'gzip', engine = 'pyarrow', index = False)
test_x.to_parquet('drive/MyDrive/data/test_v3.parquet', compression = 'gzip', engine = 'pyarrow', index = False)
train_y.to_parquet('drive/MyDrive/data/trainy_v3.parquet', compression = 'gzip', engine = 'pyarrow', index = False)

AttributeError: 'Series' object has no attribute 'to_parquet'

In [ ]:
# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [ ]:
!pip install flaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 5.5 MB/s eta 0:00:00


In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.5 MB/s eta 0:00:00


In [ ]:
from flaml import AutoML
# import wandb

In [ ]:
# Initialize AutoML
automl = AutoML()

automl_settings = {
    "time_budget": 3600,  # Total time budget in seconds
    "metric": 'roc_auc',  # Evaluation metric
    "task": 'classification',  # Task type
    "log_file_name": 'automl.log',  # Log file
    #"estimator_list": ['lgbm', 'xgboost', 'catboost', 'rf', 'extra_tree'],  # List of estimators to use
     "estimator_list": ['lgbm'],
    #"eval_method": "holdout",  # Use holdout validation method
    # "split_ratio": 0.2,  # Ratio of data to be used as validation set
    "early_stop": 10
}


In [ ]:
# Fit the model
automl.fit(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, **automl_settings)

[flaml.automl.logger: 05-30 06:18:29] {1680} INFO - task = classification
[flaml.automl.logger: 05-30 06:18:29] {1688} INFO - Data split method: stratified
[flaml.automl.logger: 05-30 06:18:29] {1691} INFO - Evaluation method: holdout
[flaml.automl.logger: 05-30 06:18:34] {1789} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl.logger: 05-30 06:18:34] {1901} INFO - List of ML learners in AutoML Run: ['lgbm']
[flaml.automl.logger: 05-30 06:18:34] {2219} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 05-30 06:18:35] {2345} INFO - Estimated sufficient time budget=6675713s. Estimated necessary time budget=6676s.
[flaml.automl.logger: 05-30 06:18:35] {2392} INFO -  at 71.6s,	estimator lgbm's best error=0.3742,	best estimator lgbm's best error=0.3742
[flaml.automl.logger: 05-30 06:18:35] {2219} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 05-30 06:18:36] {2392} INFO -  at 72.4s,	estimator lgbm's best error=0.3742,	best estimator lgbm's best error=0.374

In [ ]:
sample_submission = pd.read_csv('drive/MyDrive/data/sample_submission.csv')

# Predict probabilities on the test set
y_pred_test_proba = automl.predict_proba(test_x)[:, 1]

# Output the probabilities for test set
print(y_pred_test_proba)

In [ ]:
sample_submission['Click'] = y_pred_test_proba
sample_submission

In [ ]:
sample_submission.to_csv("drive/MyDrive/data/lgbm_prediction_v3.csv",index = False)